# Establishing a Connection - Bob
## Role: Invitee

If you want to start at Part 1 of this tutorial series go [here](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%201%20-%20Self-Sovereign%20Identity.ipynb).

In this notebook we'll be responding to an invitation from Alice through our Agents. This notebook has the following phases:

<b>Begin with the [Alice notebook](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)</b>


6. Instatiate the controller for our Agent
7. Set up a message listener running as a service on our controller
8. Paste the invitation from the Alice notebook into the invitation variable here
9. Accept the invitation

<b>Carry on at the Alice side</b>
    
14. Check if connection active


### 6. Instatiate the controller for our Agent

In [9]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


In [10]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

### 7. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using PyPubSub. The default listeners are used to update state and print logs.


In [11]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


### 8. Paste the invitation from the Alice notebook into the invitation variable here;

In [12]:
#Paste in invitation from Alice agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '3d67fb90-4e97-4c5c-bff9-add0e47a1b30', 'label': 'Alice', 'recipientKeys': ['4DLhPbSzr4tGnrN18qwLEvrF4Qvntx2z7LPyYn1rDc5Z'], 'serviceEndpoint': 'http://192.168.65.3:8020'}

### 9. Accept the invitation, then move to Alice's notebook

In [13]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


Connection Handler Called
Connection 842b811f-36df-4137-8577-15469f260f8b in State invitation
Connection Handler Called
Connection 842b811f-36df-4137-8577-15469f260f8b in State request
Connection {'their_label': 'Carol', 'invitation_key': '5NqLisZNrS2AbeBNCQHcF7hZx22Xd6aYx8vxadbdkqmJ', 'invitation_mode': 'once', 'connection_id': '842b811f-36df-4137-8577-15469f260f8b', 'request_id': 'dcd5a0ac-25fa-492c-bf37-5e00fa16aac4', 'my_did': '3jAoJUdyYtxQa6AkTbaeAm', 'accept': 'manual', 'rfc23_state': 'request-sent', 'their_role': 'inviter', 'created_at': '2023-01-02 08:52:07.421499Z', 'state': 'request', 'routing_state': 'none', 'updated_at': '2023-01-02 08:52:07.477463Z'}
Connection Handler Called
Connection 842b811f-36df-4137-8577-15469f260f8b in State response
Connection Handler Called
Connection 842b811f-36df-4137-8577-15469f260f8b in State active


### 13. Check if connection state is active

It should be if you send a trust ping from Alice's side

In [14]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Alice AGENT CONNECTION
{'their_did': 'QWLH91NGnms8E57znbMPbX', 'their_label': 'Carol', 'invitation_key': '5NqLisZNrS2AbeBNCQHcF7hZx22Xd6aYx8vxadbdkqmJ', 'invitation_mode': 'once', 'connection_id': '842b811f-36df-4137-8577-15469f260f8b', 'request_id': 'dcd5a0ac-25fa-492c-bf37-5e00fa16aac4', 'my_did': '3jAoJUdyYtxQa6AkTbaeAm', 'accept': 'manual', 'rfc23_state': 'completed', 'their_role': 'inviter', 'created_at': '2023-01-02 08:52:07.421499Z', 'state': 'active', 'routing_state': 'none', 'updated_at': '2023-01-02 08:52:22.318473Z'}
State: active


## Optional: Remove connection

In [15]:
remove = await agent_controller.connections.remove_connection(alice_id)
print(remove)

{}


In [16]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'their_did': 'CXVXVERJAYXkEVaS3EB4fT', 'their_label': 'Carol', 'invitation_key': 'AuBeW7Laq6SQzUAM5JNdwv13wdXeVDzxD3myp7euN5xq', 'invitation_mode': 'once', 'connection_id': '606a4a0e-2b3b-4f3a-b402-a20ab4713590', 'request_id': '6b7c6af7-ae20-43d5-b370-ca76ced61822', 'my_did': 'LpmiP5Lpre6u2BF76nSALw', 'accept': 'manual', 'rfc23_state': 'completed', 'their_role': 'inviter', 'created_at': '2023-01-02 08:19:47.213947Z', 'state': 'active', 'routing_state': 'none', 'updated_at': '2023-01-02 08:20:24.085479Z'}]}


## End of Tutorial

#### Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [17]:
response = await agent_controller.terminate()
print(response)

None
